# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著完成三種不同特徵類型的三種資料操作, 觀察結果
- 思考一下, 這三種特徵類型, 哪一種應該最複雜/最難處理

# [作業重點]
- 完成剩餘的八種 類型 x 操作組合 (In[6]~In[13], Out[6]~Out[13])
- 思考何種特徵類型, 應該最複雜

In [5]:
# 載入基本套件
import pandas as pd
import numpy as np

# 讀取訓練與測試資料
data_path = ''
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [6]:
display(df_train)
display(df_test)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [7]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [13]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

In [47]:
# 例 : 整數 (int) 特徵取平均 (mean)
print(f'{df[int_features].mean()}\n')
print(f'{df[int_features].max()}\n')
print(f'{df[int_features].nunique()}\n')

print(f'{df[float_features].mean()}\n')
print(f'{df[float_features].max()}\n')
print(f'{df[float_features].nunique()}\n')


Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

Pclass    3
SibSp     8
Parch     9
dtype: int64

Pclass    3
SibSp     7
Parch     8
dtype: int64

Age     29.881138
Fare    33.295479
dtype: float64

Age      80.0000
Fare    512.3292
dtype: float64

Age      98
Fare    281
dtype: int64



In [46]:
print(f'{df[object_features].mean()}\n')
print(f'{df[object_features].max()}\n')
print(f'{df[object_features].nunique()}\n')


Series([], dtype: float64)

Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64



In [51]:
# 請依序列出 三種特徵類型 (int / float / object) x 三種方法 (平均 mean / 最大值 Max / 相異值 nunique) 的其餘操作
"""
Your Code Here
"""
print(f'{df[int_features].sum()}\n')
print(f'{df[float_features].sum()}\n')
print(f'{df[object_features].sum()}\n')

print(f'{df[int_features].median()}\n')
print(f'{df[float_features].median()}\n')
print(f'{df[object_features].median()}\n')

print(f'{df[int_features].diff()}\n')
print(f'{df[float_features].diff()}\n')
print(f'{df[object_features].diff()}\n')

Pclass    3004
SibSp      653
Parch      504
dtype: int64

Age     31255.6700
Fare    43550.4869
dtype: float64

Name      Braund, Mr. Owen HarrisCumings, Mrs. John Brad...
Sex       malefemalefemalefemalemalemalemalemalefemalefe...
Ticket    A/5 21171PC 17599STON/O2. 31012821138033734503...
dtype: object

Pclass    3.0
SibSp     0.0
Parch     0.0
dtype: float64

Age     28.0000
Fare    14.4542
dtype: float64

Series([], dtype: float64)

     Pclass  SibSp  Parch
0       NaN    NaN    NaN
1      -2.0    0.0    0.0
2       2.0   -1.0    0.0
3      -2.0    1.0    0.0
4       2.0   -1.0    0.0
..      ...    ...    ...
413     0.0    0.0    0.0
414    -2.0    0.0    0.0
415     2.0    0.0    0.0
416     0.0    0.0    0.0
417     0.0    1.0    1.0

[1309 rows x 3 columns]

      Age      Fare
0     NaN       NaN
1    16.0   64.0333
2   -12.0  -63.3583
3     9.0   45.1750
4     0.0  -45.0500
..    ...       ...
413   NaN    0.2750
414   NaN  100.8500
415  -0.5 -101.6500
416   NaN    0.8000


TypeError: unsupported operand type(s) for -: 'str' and 'str'